In [1]:
# ✅ Install dependencies
!pip install pyyaml flask

In [2]:
# 🧠 Import libraries & define utility functions
import os
import yaml
import json
import sqlite3
from flask import Flask, jsonify

CONFIG_FILE = 'docker-compose.yml'
DB_FILE = 'config_data.db'
TABLE_NAME = 'config_json'

def read_config_file(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Configuration file '{path}' not found.")
    try:
        with open(path, 'r') as f:
            return yaml.safe_load(f)
    except Exception as e:
        raise IOError(f"Failed to read the file: {e}")

def extract_services(config):
    services_info = []
    services = config.get('services', {})
    for name, details in services.items():
        service = {
            'name': name,
            'image': details.get('image', 'N/A'),
            'build': details.get('build', 'N/A'),
            'ports': details.get('ports', [])
        }
        services_info.append(service)
    return services_info

def store_json_in_db(data):
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            data TEXT NOT NULL
        )
    ''')
    cursor.execute(f'INSERT INTO {TABLE_NAME} (data) VALUES (?)', (json.dumps(data),))
    conn.commit()
    conn.close()

def fetch_data_from_db():
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute(f'SELECT data FROM {TABLE_NAME} ORDER BY id DESC LIMIT 1')
    result = cursor.fetchone()
    conn.close()
    return json.loads(result[0]) if result else {}

In [3]:
# 🔁 Extract and Store Config Data
try:
    config_data = read_config_file(CONFIG_FILE)
    extracted_data = extract_services(config_data)
    store_json_in_db(extracted_data)
    print("✅ Configuration data successfully extracted and stored.")
except Exception as e:
    print(f"❌ Error: {e}")

✅ Configuration data successfully extracted and stored.


In [4]:
# 🔍 Fetch from DB (Simulate GET /config)
from pprint import pprint
data = fetch_data_from_db()
pprint(data)

[{'build': './app', 'image': 'N/A', 'name': 'web', 'ports': ['5000:5000']},
 {'build': 'N/A', 'image': 'postgres:13', 'name': 'db', 'ports': []},
 {'build': 'N/A',
  'image': 'dpage/pgadmin4',
  'name': 'pgadmin',
  'ports': ['8080:80']}]


In [ ]:
# 🚀 Optional: Run Flask API inside Notebook
app = Flask(__name__)

@app.route('/config', methods=['GET'])
def get_config():
    data = fetch_data_from_db()
    return jsonify(data)

# Non-blocking flask for notebook usage
app.run(debug=False, port=5001, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
